In [257]:
import numpy as np
import pandas as pd

In [256]:
df1 = pd.read_csv("modeling data/1_players_info_added_Clean_data.csv")

df1_nulls = df1.isnull().sum()

print(df1_nulls)

Unnamed: 0           0
Location             0
Date                 0
Round                0
Winner               0
Loser                0
WRank               60
LRank              206
W_hand               0
W_ht               957
W_nationality      858
W_age              859
L_hand               0
L_ht              1102
L_nationality      791
L_age              796
L_wt             11818
W_wt             11801
dtype: int64


In [258]:
df1_dropped_wt = df1.copy()

df1_dropped_wt.drop(columns=["Unnamed: 0",'L_wt', 'W_wt'], inplace=True)
df1_no_nulls = df1_dropped_wt.dropna()

len(df1_no_nulls)
# df1_no_nulls


17917

In [259]:
df_winner = df1_no_nulls.copy()
df_loser = df1_no_nulls.copy()

df_winner['win'] = 1
df_loser['win'] = 0

# Renaming columns to 'player' and 'opponent' for winners
df_winner.rename(columns={
    'Winner': 'player',
    'Loser': 'opponent',
    'WRank': 'player_rank',
    'LRank': 'opponent_rank',
    'W_hand': 'player_hand',
    'L_hand': 'opponent_hand',
    'W_ht': 'player_ht', 
    'W_nationality':'player_nationality', 
    'W_age':'player_age', 
    'L_ht':'opponent_ht',
    'L_nationality':'opponent_nationality', 
    'L_age':'opponent_age'
}, inplace=True)

# Renaming columns to 'player' and 'opponent' for losers
df_loser.rename(columns={
    'Winner': 'opponent',
    'Loser': 'player',
    'WRank': 'opponent_rank',
    'LRank': 'player_rank',
    'W_hand': 'opponent_hand',
    'L_hand': 'player_hand',
    'W_ht':'opponent_ht', 
    'W_nationality':'opponent_nationality', 
    'W_age':'opponent_age', 
    'L_ht':'player_ht',
    'L_nationality':'player_nationality',
    'L_age':'player_age'
}, inplace=True)

In [261]:
df_transformed = pd.concat([df_winner, df_loser], ignore_index=True)
df_transformed.to_csv('modeling data/df1_workable.csv', index=False)

In [262]:
# df_transformed = df_transformed.sample(frac=1).reset_index(drop=True)

player_peak_ranking = df_transformed.groupby('player')['player_rank'].min().reset_index()
opponent_peak_ranking = df_transformed.groupby('opponent')['opponent_rank'].min().reset_index()
player_peak_ranking

player_updated = pd.merge(df_transformed, player_peak_ranking[['player', 'player_rank']], on='player', how='left')
opponent_updated = pd.merge(player_updated, opponent_peak_ranking[['opponent', 'opponent_rank']], on='opponent', how='left')
df1 = opponent_updated.copy()
df1.drop(columns=['player_rank_x', 'opponent_rank_x'], inplace=True)
df1 = df1.rename(columns={  'player_rank_y': 'player_rank',
                            'opponent_rank_y': 'opponent_rank'})
df1

,Location,Date,Round,player,opponent,player_hand,player_ht,player_nationality,player_age,opponent_hand,opponent_ht,opponent_nationality,opponent_age,win,player_rank,opponent_rank
0,Brisbane,01/01/2017,Round of 128,Thompson J.,Ymer E.,R,183.0,AUS,24.6,R,183.0,SWE,22.8,1,43.0,108.0
1,Brisbane,01/02/2017,Round of 128,Mahut N.,Robert S.,R,190.0,FRA,37.3,R,185.0,FRA,39.6,1,38.0,53.0
2,Brisbane,01/02/2017,Round of 128,Ferrer D.,Tomic B.,R,175.0,ESP,36.7,R,193.0,AUS,26.2,1,21.0,26.0
3,Brisbane,01/02/2017,Round of 128,Edmund K.,Escobedo E.,R,188.0,GBR,24.1,R,185.0,MEX,23.1,1,14.0,69.0
4,Brisbane,01/02/2017,Round of 128,Dimitrov G.,Johnson S.,R,188.0,BUL,27.6,R,188.0,USA,29.0,1,3.0,25.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,nitto-atp-finals,01/01/2017,Round Robin,Busta P.,Thiem D.,R,188.0,ESP,33.0,R,185.0,AUT,31.0,0,15.0,3.0
35830,nitto-atp-finals,01/01/2017,Round Robin,Cilic M.,Sock J.,R,198.0,CRO,36.0,R,191.0,USA,32.0,0,3.0,8.0
35831,nitto-atp-finals,01/01/2017,Round Robin,Busta P.,Dimitrov G.,R,188.0,ESP,33.0,R,191.0,BUL,33.0,0,15.0,3.0
35832,nitto-atp-finals,01/01/2017,Round Robin,Goffin D.,Dimitrov G.,R,180.0,BEL,34.0,R,191.0,BUL,33.0,0,7.0,3.0


In [263]:
df1_home_country = df1.copy()
df1_home_country['is_player_home_country'] = np.equal(df1_home_country['Location'], df1_home_country['player_nationality'])
df1_home_country['is_opponent_home_country'] = np.equal(df1_home_country['Location'], df1_home_country['opponent_nationality'])
df1_home_country.drop(columns=["Location",'player_nationality', 'opponent_nationality'], inplace=True)
df1_home_country


,Date,Round,player,opponent,player_hand,player_ht,player_age,opponent_hand,opponent_ht,opponent_age,win,player_rank,opponent_rank,is_player_home_country,is_opponent_home_country
0,01/01/2017,Round of 128,Thompson J.,Ymer E.,R,183.0,24.6,R,183.0,22.8,1,43.0,108.0,False,False
1,01/02/2017,Round of 128,Mahut N.,Robert S.,R,190.0,37.3,R,185.0,39.6,1,38.0,53.0,False,False
2,01/02/2017,Round of 128,Ferrer D.,Tomic B.,R,175.0,36.7,R,193.0,26.2,1,21.0,26.0,False,False
3,01/02/2017,Round of 128,Edmund K.,Escobedo E.,R,188.0,24.1,R,185.0,23.1,1,14.0,69.0,False,False
4,01/02/2017,Round of 128,Dimitrov G.,Johnson S.,R,188.0,27.6,R,188.0,29.0,1,3.0,25.0,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,01/01/2017,Round Robin,Busta P.,Thiem D.,R,188.0,33.0,R,185.0,31.0,0,15.0,3.0,False,False
35830,01/01/2017,Round Robin,Cilic M.,Sock J.,R,198.0,36.0,R,191.0,32.0,0,3.0,8.0,False,False
35831,01/01/2017,Round Robin,Busta P.,Dimitrov G.,R,188.0,33.0,R,191.0,33.0,0,15.0,3.0,False,False
35832,01/01/2017,Round Robin,Goffin D.,Dimitrov G.,R,180.0,34.0,R,191.0,33.0,0,7.0,3.0,False,False


In [265]:
# df1_home_country['Date'] = pd.to_datetime(df1_home_country['Date'])

# df1_home_country['year'] = df1_home_country['Date'].dt.year
# df1_home_country['month'] = df1_home_country['Date'].dt.month
# df1_home_country['day'] = df1_home_country['Date'].dt.day
# df1_home_country['dayofweek'] = df1_home_country['Date'].dt.dayofweek
# df1_home_country.drop(columns=["Date"], inplace=True)
# df1_home_country.to_csv('modeling data/df1_with_home.csv', index=False)
df_home = pd.read_csv("modeling data/df1_with_home.csv")
df_home.columns

Index(['Round', 'player', 'opponent', 'player_hand', 'player_ht', 'player_age',
       'opponent_hand', 'opponent_ht', 'opponent_age', 'win', 'player_rank',
       'opponent_rank', 'is_player_home_country', 'is_opponent_home_country',
       'year', 'month', 'day', 'dayofweek'],
      dtype='object')

In [273]:
df_home_encoded = df_home.copy()
df_home_encoded['month_sin'] = np.sin(2 * np.pi * df_home_encoded['month']/12)
df_home_encoded['month_cos'] = np.cos(2 * np.pi * df_home_encoded['month']/12)
df_home_encoded['dayofweek_sin'] = np.sin(2 * np.pi * df_home_encoded['dayofweek']/7)
df_home_encoded['dayofweek_cos'] = np.cos(2 * np.pi * df_home_encoded['dayofweek']/7)
df_home_encoded['day_sin'] = np.sin(2 * np.pi * df_home_encoded['day']/30.5)
df_home_encoded['day_cos'] = np.cos(2 * np.pi * df_home_encoded['day']/30.5)

df_home_encoded.drop(columns=["month","dayofweek", "day"], inplace=True)
df_home_encoded

,Round,player,opponent,player_hand,player_ht,player_age,opponent_hand,opponent_ht,opponent_age,win,...,opponent_rank,is_player_home_country,is_opponent_home_country,year,month_sin,month_cos,dayofweek_sin,dayofweek_cos,day_sin,day_cos
0,Quarterfinals,Darcis S.,Young D.,R,178,34.8,L,183,35.0,0,...,42.0,False,True,2017,0.866025,5.000000e-01,-0.433884,-0.900969,-0.973264,0.229688
1,Round of 128,Marchenko I.,Jasika O.,R,185,37.0,L,183,27.0,0,...,290.0,False,False,2016,0.500000,8.660254e-01,-0.433884,-0.900969,0.204552,0.978856
2,Round of 32,Johnson S.,Dimitrov G.,R,188,35.0,R,191,33.0,1,...,3.0,False,False,2016,0.500000,8.660254e-01,-0.433884,-0.900969,0.204552,0.978856
3,Semifinals,Kohlschreiber P.,Monfils G.,R,178,41.0,R,193,38.0,0,...,6.0,False,False,2016,0.500000,8.660254e-01,-0.433884,-0.900969,0.204552,0.978856
4,Round of 128,Daniel T.,Barrere G.,R,191,27.5,R,183,26.5,0,...,82.0,False,False,2020,-0.866025,-5.000000e-01,0.000000,1.000000,0.102821,0.994700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,1st Round Qualifying,Gulbis E.,Stakhovsky S.,R,193,36.0,R,193,38.0,0,...,96.0,False,False,2017,0.500000,8.660254e-01,-0.781831,0.623490,0.204552,0.978856
35830,Round of 64,Sousa J.,Kohlschreiber P.,R,185,29.7,R,178,35.2,1,...,22.0,False,False,2019,0.500000,8.660254e-01,0.433884,-0.900969,-0.352752,-0.935717
35831,Round of 32,Goffin D.,Nishioka Y.,R,180,34.0,L,170,29.0,1,...,48.0,False,True,2016,0.500000,8.660254e-01,-0.433884,-0.900969,0.204552,0.978856
35832,Round Robin,Zverev A.,Raonic M.,R,198,22.4,R,196,28.7,1,...,3.0,False,False,2019,-1.000000,-1.836970e-16,-0.433884,-0.900969,-0.829677,-0.558244


In [277]:
df_round_encoded = df_home_encoded.copy()
round_mapping = {
    '1st Round Qualifying': 1,
    '2nd Round Qualifying': 2,
    '3rd Round Qualifying': 3,
    'Round Robin': 4,
    'Round of 128': 5,
    'Round of 64': 6,
    'Round of 32': 7,
    'Round of 16': 8,
    'Quarterfinals': 9,
    'Semifinals': 10,
    'The Final': 11
}
df_round_encoded['Round'] = df_round_encoded['Round'].map(round_mapping)

In [281]:
df_binary_encoded = df_round_encoded.copy()

df_binary_encoded['is_player_home_country'] = df_binary_encoded['is_player_home_country'].astype(int)
df_binary_encoded['is_opponent_home_country'] = df_binary_encoded['is_opponent_home_country'].astype(int)
df_binary_encoded.columns

Index(['Round', 'player', 'opponent', 'player_hand', 'player_ht', 'player_age',
       'opponent_hand', 'opponent_ht', 'opponent_age', 'win', 'player_rank',
       'opponent_rank', 'is_player_home_country', 'is_opponent_home_country',
       'year', 'month_sin', 'month_cos', 'dayofweek_sin', 'dayofweek_cos',
       'day_sin', 'day_cos'],
      dtype='object')

In [282]:
df_player_encode = df_binary_encoded.copy()
#encoding rows
def encode(og_df):
    df = og_df.copy()
    for column in df.columns:
        if column in ["player","opponent"]:
            print(column) #which columns are affected
            i = 0
            for items in df[column].unique():
                df[column].replace({items: i}, inplace=True)
                i+=1
    return df
df_player_encode = encode(df_player_encode)
df_player_encode

player
opponent


,Round,player,opponent,player_hand,player_ht,player_age,opponent_hand,opponent_ht,opponent_age,win,...,opponent_rank,is_player_home_country,is_opponent_home_country,year,month_sin,month_cos,dayofweek_sin,dayofweek_cos,day_sin,day_cos
0,9,0,0,R,178,34.8,L,183,35.0,0,...,42.0,0,1,2017,0.866025,5.000000e-01,-0.433884,-0.900969,-0.973264,0.229688
1,5,1,1,R,185,37.0,L,183,27.0,0,...,290.0,0,0,2016,0.500000,8.660254e-01,-0.433884,-0.900969,0.204552,0.978856
2,7,2,2,R,188,35.0,R,191,33.0,1,...,3.0,0,0,2016,0.500000,8.660254e-01,-0.433884,-0.900969,0.204552,0.978856
3,10,3,3,R,178,41.0,R,193,38.0,0,...,6.0,0,0,2016,0.500000,8.660254e-01,-0.433884,-0.900969,0.204552,0.978856
4,5,4,4,R,191,27.5,R,183,26.5,0,...,82.0,0,0,2020,-0.866025,-5.000000e-01,0.000000,1.000000,0.102821,0.994700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,1,192,126,R,193,36.0,R,193,38.0,0,...,96.0,0,0,2017,0.500000,8.660254e-01,-0.781831,0.623490,0.204552,0.978856
35830,6,237,32,R,185,29.7,R,178,35.2,1,...,22.0,0,0,2019,0.500000,8.660254e-01,0.433884,-0.900969,-0.352752,-0.935717
35831,7,14,102,R,180,34.0,L,170,29.0,1,...,48.0,0,1,2016,0.500000,8.660254e-01,-0.433884,-0.900969,0.204552,0.978856
35832,4,13,9,R,198,22.4,R,196,28.7,1,...,3.0,0,0,2019,-1.000000,-1.836970e-16,-0.433884,-0.900969,-0.829677,-0.558244


In [283]:
df_hand_int_encoded = df_player_encode.copy()
hand_mapping = {
    'R': 1,
    'L': 2,
    'U': 0
}
df_hand_int_encoded['player_hand'] = df_hand_int_encoded['player_hand'].map(hand_mapping)
df_hand_int_encoded['opponent_hand'] = df_hand_int_encoded['opponent_hand'].map(hand_mapping)
df_hand_int_encoded

,Round,player,opponent,player_hand,player_ht,player_age,opponent_hand,opponent_ht,opponent_age,win,...,opponent_rank,is_player_home_country,is_opponent_home_country,year,month_sin,month_cos,dayofweek_sin,dayofweek_cos,day_sin,day_cos
0,9,0,0,1,178,34.8,2,183,35.0,0,...,42.0,0,1,2017,0.866025,5.000000e-01,-0.433884,-0.900969,-0.973264,0.229688
1,5,1,1,1,185,37.0,2,183,27.0,0,...,290.0,0,0,2016,0.500000,8.660254e-01,-0.433884,-0.900969,0.204552,0.978856
2,7,2,2,1,188,35.0,1,191,33.0,1,...,3.0,0,0,2016,0.500000,8.660254e-01,-0.433884,-0.900969,0.204552,0.978856
3,10,3,3,1,178,41.0,1,193,38.0,0,...,6.0,0,0,2016,0.500000,8.660254e-01,-0.433884,-0.900969,0.204552,0.978856
4,5,4,4,1,191,27.5,1,183,26.5,0,...,82.0,0,0,2020,-0.866025,-5.000000e-01,0.000000,1.000000,0.102821,0.994700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,1,192,126,1,193,36.0,1,193,38.0,0,...,96.0,0,0,2017,0.500000,8.660254e-01,-0.781831,0.623490,0.204552,0.978856
35830,6,237,32,1,185,29.7,1,178,35.2,1,...,22.0,0,0,2019,0.500000,8.660254e-01,0.433884,-0.900969,-0.352752,-0.935717
35831,7,14,102,1,180,34.0,2,170,29.0,1,...,48.0,0,1,2016,0.500000,8.660254e-01,-0.433884,-0.900969,0.204552,0.978856
35832,4,13,9,1,198,22.4,1,196,28.7,1,...,3.0,0,0,2019,-1.000000,-1.836970e-16,-0.433884,-0.900969,-0.829677,-0.558244


In [284]:
df_hand_1hot_encoded = df_player_encode.copy()

df_hand_1hot_encoded = pd.get_dummies(df_hand_1hot_encoded, columns=['player_hand'], prefix='player_hand')
df_hand_1hot_encoded = pd.get_dummies(df_hand_1hot_encoded, columns=['opponent_hand'], prefix='opponent_hand')
df_hand_1hot_encoded

,Round,player,opponent,player_ht,player_age,opponent_ht,opponent_age,win,player_rank,opponent_rank,...,dayofweek_sin,dayofweek_cos,day_sin,day_cos,player_hand_L,player_hand_R,player_hand_U,opponent_hand_L,opponent_hand_R,opponent_hand_U
0,9,0,0,178,34.8,183,35.0,0,38.0,42.0,...,-0.433884,-0.900969,-0.973264,0.229688,0,1,0,1,0,0
1,5,1,1,185,37.0,183,27.0,0,74.0,290.0,...,-0.433884,-0.900969,0.204552,0.978856,0,1,0,1,0,0
2,7,2,2,188,35.0,191,33.0,1,25.0,3.0,...,-0.433884,-0.900969,0.204552,0.978856,0,1,0,0,1,0
3,10,3,3,178,41.0,193,38.0,0,22.0,6.0,...,-0.433884,-0.900969,0.204552,0.978856,0,1,0,0,1,0
4,5,4,4,191,27.5,183,26.5,0,69.0,82.0,...,0.000000,1.000000,0.102821,0.994700,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35829,1,192,126,193,36.0,193,38.0,0,74.0,96.0,...,-0.781831,0.623490,0.204552,0.978856,0,1,0,0,1,0
35830,6,237,32,185,29.7,178,35.2,1,35.0,22.0,...,0.433884,-0.900969,-0.352752,-0.935717,0,1,0,0,1,0
35831,7,14,102,180,34.0,170,29.0,1,7.0,48.0,...,-0.433884,-0.900969,0.204552,0.978856,0,1,0,1,0,0
35832,4,13,9,198,22.4,196,28.7,1,3.0,3.0,...,-0.433884,-0.900969,-0.829677,-0.558244,0,1,0,0,1,0


In [285]:
numeric_columns = ['player_ht', 'player_age', 'opponent_ht', 'opponent_age', 'player_rank', 'opponent_rank',"year"]


In [96]:
# from sklearn.model_selection import train_test_split

# # Assume 'df_transformed' is your DataFrame and 'win' is the target variable.

# # Define your features (X) and the target variable (y)
# X = df_transformed.drop(columns=['player', 'opponent', 'win'])  # Exclude non-numeric or target columns
# y = df_transformed['win']

# # Split the data into training and testing sets (80% train, 20% test)
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# # X_train and y_train are now your training features and target
# # X_test and y_test are your testing features and target

ModuleNotFoundError: No module named 'sklearn'